In [22]:
import dask.bag as db
from itertools import chain
import glob
from tqdm import tqdm

folder = '/data_lg/olga/maca/facs/sourmash_compute_tissue_subset/'

globber = f"{folder}/A1-*.sig"

b = db.read_text(globber)
b

dask.bag<bag-fro..., npartitions=5>

In [4]:
ls $folder/A1-*.sig

/data_lg/olga/maca/facs/sourmash_compute_tissue_subset//A1-B002764-3_38_F-1-1.sig
/data_lg/olga/maca/facs/sourmash_compute_tissue_subset//A1-D042253-3_9_M-1-1.sig
/data_lg/olga/maca/facs/sourmash_compute_tissue_subset//A1-MAA000487-3_10_M-1-1.sig
/data_lg/olga/maca/facs/sourmash_compute_tissue_subset//A1-MAA000779-3_11_M-1-1.sig
/data_lg/olga/maca/facs/sourmash_compute_tissue_subset//A1-MAA100039-3_11_M-1-1.sig


In [49]:
%%time

import json

%time b_json_loads = db.read_text(globber).map(json.loads)
b_json_loads

%time b_json_loads_loaded = list(chain(*b_json_loads))
single_sig_json = b_json_loads_loaded[0]
single_sig_json

CPU times: user 6.18 ms, sys: 13 µs, total: 6.2 ms
Wall time: 4.92 ms
CPU times: user 8min 3s, sys: 6.3 s, total: 8min 9s
Wall time: 11min 18s
CPU times: user 8min 3s, sys: 6.3 s, total: 8min 9s
Wall time: 11min 18s


In [53]:

with open('/data_lg/olga/maca/facs/sourmash_compute_tissue_subset//A1-B002764-3_38_F-1-1.sig') as f:
    json_signatures = json.load(f)
json_signatures[0].keys()

dict_keys(['class', 'email', 'filename', 'hash_function', 'license', 'name', 'signatures', 'version'])

In [54]:
len(json_signatures)

1

In [61]:
from IPython.lib.pretty import pprint
pprint(json_signatures[0], max_seq_length=10)

{'class': 'sourmash_signature',
 'email': '',
 'filename': '/arg/2/1',
 'hash_function': '0.murmur64',
 'license': 'CC0',
 'name': 'A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell',
 'signatures': [{'abundances': [1, 1, 1, 1, 1, 1, 1, 2, 1, 3, ...],
   'ksize': 21,
   'max_hash': 0,
   'md5sum': '15e24523b2f4fc4b809202f4edc752e4',
   'mins': [5374179861363,
    7024158594514,
    7773053314393,
    9076196387442,
    17019727878151,
    21268452495739,
    22092088695955,
    26181274510539,
    27371326608036,
    29243034423266,
    ...],
   'molecule': 'protein',
   'num': 1000000,
   'seed': 42},
  {'abundances': [1, 6, 1, 2, 1, 1, 1, 1, 1, 2, ...],
   'ksize': 21,
   'max_hash': 0,
   'md5sum': '89bb22ab62c56c42abd4131e13d3231d',
   'mins': [4319888920071,
    11897683056067,
    15688568034930,
    18288931836992,
    27514404317028,
    28774406637780,
    28801856705165,
    2969595548

In [ ]:
{"namespace": "sourmash.signature",
 "type": "record",
 "name": "Sample",
 "fields": [
     {"name": "name", "type": "string"},
     {"name": "class",  "type": "string"},
     {"name": "email", "type": ["string", "null"]}
     {"name": "filename",  "type": "string"},
     {"name": "hash_function",  "type": "string"},
     {"name": "license",  "type": "string"},
     {"name": "signatures"}
 ] 
}

In [ ]:
b.to_avro()

In [8]:
from sourmash import signature as sig
sig.load_signatures?

In [10]:
b = db.read_text(globber).map(sig.load_signatures)
b

dask.bag<map-loa..., npartitions=5>

In [11]:
b?

In [13]:

list(b)

[[SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 15e24523),
  SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 89bb22ab),
  SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 7c03cbd2),
  SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 162db048),
  SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 73c77f17),
  SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_

In [15]:
%%time
%time b = db.read_text(globber).map(sig.load_signatures)
%time sigs = list(chain(*b))

CPU times: user 5.32 ms, sys: 0 ns, total: 5.32 ms
Wall time: 4.54 ms
CPU times: user 9min 27s, sys: 4.7 s, total: 9min 32s
Wall time: 15min 9s
CPU times: user 9min 27s, sys: 4.7 s, total: 9min 32s
Wall time: 15min 9s


In [24]:
%%time

filenames = glob.glob(globber)
print(f'len(filenames): {len(filenames)}')

siglist_traditional = []

for filename in tqdm(filenames):
    loaded = sig.load_signatures(filename)
    siglist_traditional.extend(loaded)

  0%|          | 0/5 [00:00<?, ?it/s]

len(filenames): 5


100%|██████████| 5/5 [12:35<00:00, 150.98s/it]

CPU times: user 12min 33s, sys: 3.12 s, total: 12min 36s
Wall time: 12min 35s


In [ ]:
%%time

siglist_traditional_chained = list(chain(*[sig.load_signatures(f) for f in tqdm(filenames)]))

100%|██████████| 5/5 [00:00<00:00, 40960.00it/s]


In [28]:
siglist_traditional_chained[:5]

[SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 15e24523),
 SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 89bb22ab),
 SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 7c03cbd2),
 SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 162db048),
 SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 73c77f17)]

In [29]:
sigs[:5]

[SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 15e24523),
 SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 89bb22ab),
 SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 7c03cbd2),
 SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 162db048),
 SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 73c77f17)]

In [30]:
b

dask.bag<bag-fro..., npartitions=5>

In [33]:
! tree -d /data_lg/olga/maca/

/data_lg/olga/maca/
└── facs
    └── sourmash_compute_tissue_subset

2 directories


In [35]:
folder = '/data_lg/olga/maca/facs/sourmash_compute_tissue_subset/dask'
! mkdir $folder

In [37]:
s = sigs[0]

In [39]:
s

SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 15e24523)

In [40]:
s.d

{'class': 'sourmash_signature',
 'license': 'CC0',
 'name': 'A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell',
 'filename': '/arg/2/1'}

In [41]:
s.__dict__

{'d': {'class': 'sourmash_signature',
  'license': 'CC0',
  'name': 'A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell',
  'filename': '/arg/2/1'},
 'minhash': <sourmash._minhash.MinHash at 0x7fff8d9c5198>}

In [46]:
dir(s.minhash)

['__class__',
 '__copy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'add',
 'add_hash',
 'add_many',
 'add_protein',
 'add_sequence',
 'compare',
 'contained_by',
 'containment_ignore_maxhash',
 'copy_and_clear',
 'count_common',
 'downsample_max_hash',
 'downsample_n',
 'downsample_scaled',
 'get_hashes',
 'get_mins',
 'intersection',
 'is_molecule_type',
 'is_protein',
 'jaccard',
 'ksize',
 'max_hash',
 'merge',
 'num',
 'remove_many',
 'scaled',
 'seed',
 'set_abundances',
 'similarity',
 'subtract_mins',
 'track_abundance',
 'update']

In [ ]:
schema = {'name': 'Test',
 'namespace': 'Test',
 'doc': 'Descriptive text',
 'type': 'record',
 'fields': [
    {'name': 'a', 'type': 'int'},
 ]}


In [36]:
db.avro.to_avro(b, f'{folder}/A1_signatures.avro')

TypeError: to_avro() missing 1 required positional argument: 'schema'